In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer


In [ ]:
df = pd.read_excel("aug_nine_var.xlsx")
X = df.loc[:,["국내총생산", "서울인구", "경기도인구", "인천시인구", "수도권 인구", "소비자 물가 지수"]].to_numpy()
y = df.iloc[:,22:23].to_numpy()


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
# encoding dimension
encoding_dim = 32
 
input_layer = tf.keras.layers.Input(shape=(6,)) # 28*28
encoded = tf.keras.layers.Dense(32, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(64, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(128)(encoded)
 
decoded = tf.keras.layers.Dense(128, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(64, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(32, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(6)(decoded)
 
# autoencoder
autoencoder = tf.keras.Model(input_layer, decoded)
 
# encoder
encoder = tf.keras.Model(input_layer, encoded)
 
# decoder
encoded_input = tf.keras.layers.Input(shape=(128,))
decoded_layer = autoencoder.layers[4](encoded_input)
decoded_layer = autoencoder.layers[5](decoded_layer)
decoded_layer = autoencoder.layers[6](decoded_layer)
decoded_layer = autoencoder.layers[7](decoded_layer)
decoder = tf.keras.Model(encoded_input, decoded_layer)
 
autoencoder.compile(optimizer='adam', loss='mse', metrics=['mae'])


In [ ]:
hist = autoencoder.fit(X_train, y_train, 
                batch_size=64, epochs=10000,
                validation_data=(X_test,y_test),
                verbose=1)

In [ ]:
fig, loss_ax = plt.subplots(figsize=(15, 5))

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train_mse')   # 훈련데이터의 loss (즉, mse)
loss_ax.plot(hist.history['val_loss'], 'r', label='test_mse') # 검증데이터의 loss (즉, mse)

acc_ax.plot(hist.history['mae'], 'b', label='train_mae')   # 훈련데이터의 mae
acc_ax.plot(hist.history['val_mae'], 'g', label='val_mae') # 검증데이터의 mae

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('mean_absolute_error')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()